In [211]:
import os
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
import json
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu,SmoothingFunction
import pandas as pd
import numpy as np
import docker
import os
import re
import random
from string import punctuation
import json
import matplotlib.pyplot as plt
import pickle
punc = punctuation
smooth = SmoothingFunction()

In [272]:
#countGasRes
def countGasRes(data):
    #patNum = [1,6,2,3,4,7] 雀屏论文顺序有误 这个顺序对应1，2，3，4，5，6
    res = []
    for idx,row in data.iterrows():
        pat = []
        for item in [1,6,2,3,4,7]:
            temp = row['pattern{}'.format(item)]
            if temp != None and len(temp) > 0:
                pat.append({
                    'pat':item,
                    'loc':temp})
        if len(pat) == 0:
            continue
        res.append({
            'row':row[['contract_code','sni_code']],
            'res': pat
        }) 
    return res

allToolNames = ['mythril_analysis_not_info','osiris_analysis','slither_analysis_not_info']

def get(cc, toolNames):
    accRes = []
    for index,row in cc.iterrows():
        #同行算消除
        res = {}
        for toolName in toolNames:
            #toolRes = row[toolName]
            toolRes =  eval(row[toolName]) if type(row[toolName]) == str else row[toolName]
            for tRes in toolRes:
                lineNum = tRes['lineno']
                #title = mapp[toolName.split('_')[0]][tRes['title'].strip()]#已经转换过
                title = tRes['title']#已经转换过
                if title in res.keys():
                    updateMap(res, title, lineNum)
                else:
                    res[title] = [[lineNum] if type(lineNum) == int else lineNum]#使用lineNum--list的len计算数量 [[],[]]
        if len(res) > 0:
            accRes.append({
                'row':row[['contract_code','sni_code']],
                'res':res
            })
    return accRes

def gasAccCheck(g):
    loc = []
    error = []
    contract = []
    sni = []
    bleu = []
    for item in g:
        for i in item['res']:#pat内存储多个结果
            #for l in i['loc']:
            loc.append(i['loc'])
            error.append(i['pat'])
            contract.append(re.sub(r'\n[\s]*\n', '\n', item['row'][0]))
            sni.append(re.sub(r'\n[\s]*\n', '\n', item['row'][1]))
            bleu.append(sentence_bleu([item['row'][1]],item['row'][0],smoothing_function=smooth.method1))
    gas = pd.DataFrame({
        'error':error,
        'loc':loc,
        'contract':contract,
        'sni':sni,
        'bleu':bleu
    })
    return gas

def bugAccCheck(g):
    loc = []
    error = []
    contract = []
    sni = []
    bleu = []
    for item in g:
        for k in item['res'].keys():#pat内存储多个结果
                for l in item['res'][k]:
                    loc.append(l)
                    error.append(k)
                    contract.append(re.sub(r'\n[\s]*\n', '\n', item['row'][0]))
                    sni.append(re.sub(r'\n[\s]*\n', '\n', item['row'][1]))
                    bleu.append(sentence_bleu([item['row'][1]],item['row'][0],smoothing_function=smooth.method1))
    gas = pd.DataFrame({
        'error':error,
        'loc':loc,
        'contract':contract,
        'sni':sni,
        'bleu':bleu
    })
    return gas

def getErrorPair(data,t):
    return data[data[t].apply(lambda x : x != None)]

col = ['pattern6','pattern2','pattern3','pattern4','pattern7','slither_analysis_not_info','mythril_analysis_not_info','osiris_analysis']
col0 = 'pattern1'

In [199]:
def getPair(data):
    temp = getErrorPair(data, col0)
    for item in col:
        temp.append(getErrorPair(data, item))
    temp = temp['index'].drop_duplicates()
    return data[data['index'].apply(lambda x : x in temp)]

In [205]:
ttt = getPair(conAst)

In [293]:
nicadF8 = pd.read_pickle('/home/yfliu/paper_data/astResultCsv/functionRes_nicad3_0.8.pic')
nicadC8 = pd.read_pickle('/home/yfliu/paper_data/astResultCsv/contractRes_nicad3_0.8.pic')

In [292]:
len(nicadF5)

155

In [282]:
nicadF = pd.read_pickle('/home/yfliu/paper_data/astResultCsv/functionRes_nicad32C.pic')
nicadC = pd.read_pickle('/home/yfliu/paper_data/astResultCsv/contractRes_nicad32C.pic')

In [283]:
print(len(getPair(nicadC)))
print(len(getPair(nicadF)))

81
18


In [276]:
gasC = gasAccCheck(countGasRes(nicadC)).sample(100)
gasC.to_csv('/home/yfliu/paper_data/acc/nicad3ConGasAcc_nicad3.csv')
bugC = bugAccCheck(get(nicadC,allToolNames)).sample(100)
bugC.to_csv('/home/yfliu/paper_data/acc/nicad3ConBugAcc_nicad3.csv')
gasF = gasAccCheck(countGasRes(nicadF))
gasF = gasF.sample(min(len(gasF), 100))
gasF.to_csv('/home/yfliu/paper_data/acc/nicad3FunGasAcc_nicad3.csv')
bugF = bugAccCheck(get(nicadF,allToolNames))
bugF = bugF.sample(min(len(bugF), 100))
bugF.to_csv('/home/yfliu/paper_data/acc/nicad3FunBugAcc_nicad3.csv')

In [235]:
print(len(getPair(nicadC)))
print(len(getPair(nicadF)))

81
18


In [277]:
gasC = gasAccCheck(countGasRes(nicadC)).sample(100)
gasC.to_csv('/home/yfliu/paper_data/acc/conAstGasAcc_v2.csv')
bugC = bugAccCheck(get(nicadC,allToolNames)).sample(100)
bugC.to_csv('/home/yfliu/paper_data/acc/conAstBugAcc_v2.csv')
gasF = gasAccCheck(countGasRes(nicadF))
gasF = gasF.sample(min(len(gasF), 100))
gasF.to_csv('/home/yfliu/paper_data/acc/funAstGasAcc_v2.csv')
bugF = bugAccCheck(get(nicadF,allToolNames))
bugF = bugF.sample(min(len(bugF), 100))
bugF.to_csv('/home/yfliu/paper_data/acc/funAstBugAcc_v2.csv')

In [278]:
print(len(getPair(conAst)))
print(len(getPair(funAst)))

180
198


In [279]:
nicad2F =pd.read_pickle( '/home/yfliu/paper_data/astResultCsv/functionRes_nicad.pic')
nicad2C = pd.read_pickle('/home/yfliu/paper_data/astResultCsv/contractRes_nicad.pic')

In [281]:
print(len(getPair(nicad2C)))
print(len(getPair(nicad2F)))

51
11


In [245]:
manC = pd.read_pickle('/home/yfliu/paper_data/manualResultCsv/contractResGas.pic')
manF = pd.read_pickle('/home/yfliu/paper_data/manualResultCsv/functionResGas.pic')

In [246]:
print(len(getPair(manC)))
print(len(getPair(manF)))

139
39


In [243]:
getPair(funAst)[['contract_code','sni_code']]

,contract_code,sni_code
0,function TCToken(\n\n uint256 initialSu...,function MyToken(\n uint256 initialSupp...
17,function () payable {\n if (crowdsaleCl...,function () payable {\n if (crowdsaleCl...
18,function checkGoalReached() afterDeadline {\n\...,function checkGoalReached() afterDeadline {\n ...
19,function safeWithdrawal() afterDeadline {\n\n ...,function safeWithdrawal() afterDeadline {\n ...
20,function Crowdsale(\n\n address ifSucce...,function Crowdsale(\n address ifSuccessfulS...
...,...,...
597,function setUint(uint value) public {\n ...,function withdraw(uint amount) public{\n ...
602,function destroycontract(address _to) {\n\n ...,function pay (address _payee) \n {\n ...
603,function ()\n\n public\n\n payab...,function passThrough() payable public\n {\n...
604,function attach(Presale _presale){\n\t pres...,function(instance) {\n my_contract_...


# manual statisc

In [58]:
rootDir = '/home/yfliu/manual_inspection/'
conPathList = ['/home/yfliu/manual_inspection/after/segment_0.8.csv','/home/yfliu/manual_inspection/after/segment_0.9.csv','/home/yfliu/manual_inspection/after/segment_0.7.csv']
funPathList = ['/home/yfliu/manual_inspection/after_function/segment_0.8.csv','/home/yfliu/manual_inspection/after_function/segment_0.9.csv']

In [294]:
conMan = pd.read_pickle('/home/yfliu/paper_data/astResultCsv/contractRes_manual.pic')
funMan = pd.read_pickle('/home/yfliu/paper_data/astResultCsv/functionRes_manual.pic')

In [298]:
conMan = pd.read_pickle('/home/yfliu/paper_data/manualResultCsv/contractResGas.pic')
funMan = pd.read_pickle('/home/yfliu/paper_data/manualResultCsv/functionResGas.pic')

In [ ]:
def check(data):
    print(len(data[data['check'].apply(lambda x :x == 0)]))
    print(len(data[data['check'].apply(lambda x :0 < x <= 1)]))
    print(len(data[data['check'].apply(lambda x :1 < x <= 2.0)]))
check(conMan)
check(funMan)

In [60]:
def count(pList):
    x = 0
    y = 0
    for path in pList:
        data = pd.read_csv(path)
        x += len(data[data['check'].apply(lambda x : x <= 2)])
        y += len(data)
    print(x/y)#有效比例
    print(x)#数量
    print(y)

In [239]:
conAst = pd.read_pickle('/home/yfliu/paper_data/astResultCsv/contractRes_v2.pic')
funAst = pd.read_pickle('/home/yfliu/paper_data/astResultCsv/functionRes_v2.pic')

In [61]:
#查全100%
#有效性
count(conPathList)
count(funPathList)
#数量在countRes.ipynb

0.423728813559322
650
1534
0.44866920152091255
118
263


# nicad statisc

In [295]:
funN2 = pd.read_pickle('/home/yfliu/paper_data/astResultCsv/functionRes_nicad.pic')
conN2 = pd.read_pickle( '/home/yfliu/paper_data/astResultCsv/contractRes_nicad.pic')
funN3 = pd.read_pickle('/home/yfliu/paper_data/astResultCsv/functionRes_nicad32C.pic')
conN3 = pd.read_pickle('/home/yfliu/paper_data/astResultCsv/contractRes_nicad32C.pic')
funN3half = pd.read_pickle('/home/yfliu/paper_data/astResultCsv/functionRes_nicad3_0.5.pic')
conN3half = pd.read_pickle('/home/yfliu/paper_data/astResultCsv/contractRes_nicad3_0.5.pic')
funN38 = pd.read_pickle('/home/yfliu/paper_data/astResultCsv/functionRes_nicad3_0.8.pic')
conN38 = pd.read_pickle('/home/yfliu/paper_data/astResultCsv/contractRes_nicad3_0.8.pic')

In [86]:
funAST = pd.read_pickle('/home/yfliu/paper_data/astResultCsv/functionRes.pic')
conAST = pd.read_pickle('/home/yfliu/paper_data/astResultCsv/contractRes.pic')

In [68]:
funN38 = pd.read_pickle('/home/yfliu/paper_data/astResultCsv/functionRes_nicad3_0.8.pic')
conN38 = pd.read_pickle('/home/yfliu/paper_data/astResultCsv/contractRes_nicad3_0.8.pic')

In [299]:
manSniCon = list(conMan['sni_code'])
manSniFun = list(funMan['sni_code'])

In [43]:
conN3half['sni_code'].loc[2] in list(manSniCon)

True

In [ ]:
#改手工数据集为一个bug能对应的数据集 ？ 找到包含漏洞或者gas的StackOverflow代码片段 改数据集 查全指缺陷查全率 帖子查全率
#验证找到之外的结果，修改手工数据集 1
#验证准确率  1
#改到bleu>0.7结果？改代码 写错了 2

In [ ]:
#验证准确性的函数 -> 看一下低bleu情况 对额外的去验证
#补召回数据集
#提取有error结果 -》done
#修改bleu数据集？ 改为error数据集

In [300]:
def getRecall(nicad, man):
    #num = len(nicad[nicad['sni_code'].apply(lambda x : x in man)])
    #print(num, end = ' ')
    print(len(nicad[nicad['sni_code'].apply(lambda x : x in man)]) / len(man),end = ' ')
    print(len(nicad[nicad['sni_code'].apply(lambda x : x in man)]) / len(nicad))
getRecall(conN2, manSniCon)
getRecall(conN3, manSniCon)
getRecall(conN3half, manSniCon)
getRecall(conN38, manSniCon)
getRecall(conAST, manSniCon)
print('func')
getRecall(funN2, manSniFun)
getRecall(funN3, manSniFun)
getRecall(funN3half, manSniFun)
getRecall(funN38, manSniFun)
getRecall(funAST, manSniFun)

0.3212045169385194 0.9446494464944649
0.4178168130489335 0.7798594847775175
0.4629861982434128 0.5694444444444444
0.4993726474278545 0.14779056813962124
0.7452948557089084 0.5812133072407045
func
0.0 0.0
0.0 0.0
0.0 0.0
0.022727272727272728 0.01935483870967742
0.20454545454545456 0.04695652173913043


In [103]:
conAST

,index,sni_index,code_index,code_name,dealed_code,sni_name,contract_code,sni_code,contract_line,name,...,mythril_analysis,slither_analysis,pattern1,pattern6,pattern2,pattern3,pattern4,pattern7,slither_analysis_not_info,mythril_analysis_not_info
0,0,2,46,0xcf2b19384ea3f027f8620db3f797ae61bb475125.sol,"['contract', 'multisig', 'event', 'Depositaddr...",7089-1.sol,contract multisig {\n\n\n\n \n\n\n\n \n\...,contract multisig { event Deposit(address from...,"[570, 610]",0xcf2b19384ea3f027f8620db3f797ae61bb475125,...,[],[],None,None,None,None,None,None,[],[]
1,1,8,993,0x002842529757eab873cce9c251087e1b993f9200.sol,"['contract', 'StupidToken', 'function', 'trans...",37927-0.sol,contract StupidToken {\n function transferF...,contract MyToken {\n function transferF...,"[33, 35]",0x002842529757eab873cce9c251087e1b993f9200,...,[],[],None,[],None,None,None,None,[],[]
2,2,6238,993,0x002842529757eab873cce9c251087e1b993f9200.sol,"['contract', 'StupidToken', 'function', 'trans...",37858-0-0.sol,contract StupidToken {\n function transferF...,contract TheTokenContract {\n \n functio...,"[33, 35]",0x002842529757eab873cce9c251087e1b993f9200,...,[],[],None,[],None,None,None,None,[],[]
3,3,29,23020,0x59f243d056ada87eb563c2d507fe4822f8ccccac.sol,"['contract', 'MSD4', 'function', 'greenOnFanCh...",84331-2.sol,contract MSD4 {\n\n \n\n function greenOn...,contract Tested1 {\n function func() public...,"[36, 46]",0x59f243d056ada87eb563c2d507fe4822f8ccccac,...,[],[],None,[],None,None,None,None,[],[]
4,4,1745,23020,0x59f243d056ada87eb563c2d507fe4822f8ccccac.sol,"['contract', 'MSD4', 'function', 'greenOnFanCh...",84331-0.sol,contract MSD4 {\n\n \n\n function greenOn...,contract Tested {\n function func() public ...,"[36, 46]",0x59f243d056ada87eb563c2d507fe4822f8ccccac,...,[],[],None,[],None,None,None,None,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,1115,7049,2076,0x4333defa1bf3b49246666d1dea395f98e816a032.sol,"['contract', 'BitQueen', 'is', 'ERC20', 'strin...",66012-0-0.sol,contract BitQueen is ERC20 {\n\n string publi...,contract SimpleToken is ERC20 {\n\n string pu...,"[310, 328]",0x4333defa1bf3b49246666d1dea395f98e816a032,...,[],[],None,"[[317, 317]]",None,None,None,None,[],[]
1018,1116,7060,30455,0xf4c13de243a7716fb7351736228c8337f460d760.sol,"['contract', 'DreamCarToken', 'function', 'get...",71855-1-0.sol,contract DreamCarToken {\n\n function getFo...,contract A {\n function someMethod(address ...,"[16, 20]",0xf4c13de243a7716fb7351736228c8337f460d760,...,[],[],None,[],None,[],None,None,[],[]
1019,1117,7082,3571,0x5f002910d3e1bdcdcef62cbcc2b294618e9f7fad.sol,"['contract', 'TokenPriveProviderInterface', 'f...",20864-1-0.sol,contract TokenPriveProviderInterface {\n\n ...,contract Storage {\n function getEntityCoun...,"[86, 90]",0x5f002910d3e1bdcdcef62cbcc2b294618e9f7fad,...,[],[],None,"[[89, 89]]",None,None,None,None,[],[]
1020,1118,7093,459,0xc849a5fff9befa297d1c3a772bafac9f3c9004ec.sol,"['contract', 'ERC223ReceivingContract', 'funct...",34224-0-0.sol,contract ERC223ReceivingContract {\n\n\n\n ...,contract ERC223Receiver\n{\n function token...,"[38, 56]",0xc849a5fff9befa297d1c3a772bafac9f3c9004ec,...,[],[],None,"[[55, 55]]",None,None,None,None,[],[]
